<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NMF-topic-modelling-for-short-text" data-toc-modified-id="NMF-topic-modelling-for-short-text-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NMF topic modelling for short text</a></span></li></ul></div>

# NMF topic modelling for short text 



In [7]:
import os 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer 
import numpy as np
import argparse
from utils import *
import easydict
import re

import time
import argparse
import numpy as np

from utils import *
from model import *
os.chdir(r'C:\Users\niall\Dropbox\Data science masters\IN3061 INM430 Prin of Data Sc\Coursework\Documentation\SeaNMF-master')

In [56]:
os.listdir()

['.ipynb_checkpoints',
 '2009jan_2_jun_topics_no_text_preprocessing.xlsx',
 'data',
 'data_process.py',
 'LICENSE',
 'model.py',
 'NMF Short text topic modelling.ipynb',
 'README.md',
 'seanmf_results',
 'train.py',
 'utils.py',
 'vis_topic.py',
 '__pycache__',
 '~$2009jan_2_jun_topics_no_text_preprocessing.xlsx']

In [5]:
def NMF_Pre_process(text_file_loc,corpus_file_loc,vocab_file_loc,vocab_max_size):
    

    '''
    Visualize Topics
    '''


    #parser = argparse.ArgumentParser()
    #parser.add_argument('--text_file', default='data/data.txt', help='input text file')
    #parser.add_argument('--corpus_file', default='data/doc_term_mat.txt', help='term document matrix file')
    #parser.add_argument('--vocab_file', default='data/vocab.txt', help='vocab file')
    #parser.add_argument('--vocab_max_size', type=int, default=10000, help='maximum vocabulary size')
    #parser.add_argument('--vocab_min_count', type=int, default=3, help='minimum frequency of the words')
    #args = parser.parse_args()


    import easydict

    args=easydict.EasyDict({'text_file':text_file_loc,#'data/2009_jan2Jun_abc_news_text_lemma_notepad.txt', #Parameters that are inputted into thte model prior to starting processing
                            'corpus_file':corpus_file_loc,#'data/doc_term_mat_abcnews.txt',
                            'vocab_file':vocab_file_loc,#'data/vocab.txt',
                            'vocab_max_size':vocab_max_size,#10000,
                            'vocab_min_count':3
                            })

    # create vocabulary
    print('create vocab')
    vocab = {}
    fp = open(args.text_file, 'r')
    for line in fp:
        arr = re.split('\s', line[:-1]) 
        for wd in arr:
            try:
                vocab[wd] += 1
            except:
                vocab[wd] = 1 #Creating complete vocabulary list 
    fp.close()
    vocab_arr = [[wd, vocab[wd]] for wd in vocab if vocab[wd] > args.vocab_min_count] #Creating a dictionary from text file 
    vocab_arr = sorted(vocab_arr, key=lambda k: k[1])[::-1]
    vocab_arr = vocab_arr[:args.vocab_max_size]
    vocab_arr = sorted(vocab_arr)

    fout = open(args.vocab_file, 'w') #writing dictionary to vocab text file
    for itm in vocab_arr:
        itm[1] = str(itm[1])
        fout.write(' '.join(itm)+'\n')
    fout.close()

    # vocabulary to id
    vocab2id = {itm[1][0]: itm[0] for itm in enumerate(vocab_arr)}
    print('create document term matrix')
    data_arr = []
    fp = open(args.text_file, 'r') #Opens up text file and is generating a document term matrix
    fout = open(args.corpus_file, 'w')
    for line in fp: 
        arr = re.split('\s', line[:-1]) 
        arr = [str(vocab2id[wd]) for wd in arr if wd in vocab2id]
        sen = ' '.join(arr)
        fout.write(sen+'\n')
    fp.close()
    fout.close()

In [6]:
NMF_Pre_process('data/2009_jan2Jun_abc_news_text_lemma_notepad.txt','data/doc_term_mat_abcnews.txt','data/vocab.txt',10000)

 #   args=easydict.EasyDict({'text_file':text_file_loc,#'data/2009_jan2Jun_abc_news_text_lemma_notepad.txt', #Parameters that are inputted into thte model prior to starting processing
  #                          'corpus_file':corpus_file_loc,#'data/doc_term_mat_abcnews.txt',
   #                         'vocab_file':vocab_file_loc,#'data/vocab.txt',
    #                        'vocab_max_size':vocab_max_size,#10000,

create vocab
create document term matrix


In [8]:
def NMF_training_model(corpus_file_loc,vocab_file_loc,max_iter,n_topics):
    
    '''
    SeaNMF Training
    '''


    #parser = argparse.ArgumentParser()
    #parser.add_argument('--corpus_file', default='data/doc_term_mat.txt', help='term document matrix file')
    #parser.add_argument('--vocab_file', default='data/vocab.txt', help='vocab file')
    #parser.add_argument('--model', default='seanmf', help='nmf | seanmf')
    #parser.add_argument('--max_iter', type=int, default=500, help='max number of iterations')
    #parser.add_argument('--n_topics', type=int, default=100, help='number of topics')
    #parser.add_argument('--alpha', type=float, default=0.1, help='alpha')
    #parser.add_argument('--beta', type=float, default=0.0, help='beta')
    #parser.add_argument('--max_err', type=float, default=0.1, help='stop criterion')
    #parser.add_argument('--fix_seed', type=bool, default=True, help='set random seed 0')
    #args = parser.parse_args()

    args=easydict.EasyDict({'corpus_file':corpus_file_loc, #'data/doc_term_mat_abcnews.txt', #Loading model attributes into a single file 
                            'vocab_file':vocab_file_loc,#'data/vocab.txt',
                            'model':'seanmf',
                            'max_iter':max_iter,#100,
                            'n_topics':n_topics,#300,
                            'alpha':0.1,
                            'beta':0.0,
                            'max_err':0.1,
                            'fix_seed':True
                            })

    docs = read_docs(args.corpus_file) #Reading in text corpus file 
    vocab = read_vocab(args.vocab_file) #Reading in vocabulary file
    n_docs = len(docs) #Determining the total number of documents 
    n_terms = len(vocab) #Determining the total number of terms
    print('n_docs={}, n_terms={}'.format(n_docs, n_terms))

    tmp_folder = 'seanmf_results'
    if not os.access(tmp_folder, os.F_OK): #accesining folder seanmf_results to store final results in text format in
        os.mkdir(tmp_folder)

    if args.model.lower() == 'nmf': 
        print('read term doc matrix')
        dt_mat = np.zeros([n_terms, n_docs]) #initialising empty term document matrix to be filled during processing
        for k in range(n_docs):
            for j in docs[k]: # filling counts of each word in term document matrix 
                dt_mat[j, k] += 1.0
        print('term doc matrix done')
        print('-'*50)

        model = NMF( #generating NMF model with parameters term document matrix, max error, max number of toics and max number of iterations
            dt_mat, 
            n_topic=args.n_topics, 
            max_iter=args.max_iter, 
            max_err=args.max_err)

        model.save_format( #Save model in temporary folder
            Wfile=tmp_folder+'/W.txt',
            Hfile=tmp_folder+'/H.txt')

    if args.model.lower() == 'seanmf': #Generate co-occurance matrix 
        print('calculate co-occurance matrix')
        dt_mat = np.zeros([n_terms, n_terms]) #generate empty co-occurence to store data into. 
        for itm in docs:
            for kk in itm:
                for jj in itm:
                    dt_mat[int(kk),int(jj)] += 1.0 #Adding value each time cooccurence is denoted between words 
        print('co-occur done')
        print('-'*50)
        print('calculate PPMI')
        D1 = np.sum(dt_mat) #PMI value calculated between words in topic to determine the give an indicator of cooccurence of these values 
        SS = D1*dt_mat
        for k in range(n_terms):
            SS[k] /= np.sum(dt_mat[k])
        for k in range(n_terms):
            SS[:,k] /= np.sum(dt_mat[:,k])
        dt_mat = [] # release memory
        SS[SS==0] = 1.0
        SS = np.log(SS)
        SS[SS<0.0] = 0.0
        print('PPMI done')
        print('-'*50)

        print('read term doc matrix') #Size of term document matrix primary cause for memory error please ensure text corpus<1800kB
        dt_mat = np.zeros([n_terms, n_docs])
        for k in range(n_docs):
            for j in docs[k]: #Fill in term document matrix into a numpy array 
                dt_mat[j, k] += 1.0
        print('term doc matrix done')
        print('-'*50)

        model = SeaNMFL1(
            dt_mat, SS,  
            alpha=args.alpha, 
            beta=args.beta, 
            n_topic=args.n_topics, 
            max_iter=args.max_iter, 
            max_err=args.max_err,
            fix_seed=args.fix_seed)

        model.save_format(
            W1file=tmp_folder+'/W.txt',
            W2file=tmp_folder+'/Wc.txt',
            Hfile=tmp_folder+'/H.txt')



In [9]:
NMF_training_model('data/doc_term_mat_abcnews.txt','data/vocab.txt',10,10)

read documents
--------------------------------------------------
read vocabulary
--------------------------------------------------
n_docs=37996, n_terms=5999
calculate co-occurance matrix
co-occur done
--------------------------------------------------
calculate PPMI
PPMI done
--------------------------------------------------
read term doc matrix
term doc matrix done
--------------------------------------------------
loop begin
Step=0, Loss=317385.16532171227, Time=20.73029851913452s
Step=1, Loss=315124.3182260943, Time=33.059083700180054s
Step=2, Loss=313561.04908366886, Time=41.877951860427856s
Step=3, Loss=311723.41994781, Time=49.32042217254639s
Step=4, Loss=310804.23762175057, Time=57.22863554954529s
Step=5, Loss=310392.52084015845, Time=66.21713638305664s
Step=6, Loss=310198.67819562356, Time=72.68447279930115s
Step=7, Loss=310106.3264020791, Time=79.13097739219666s
Step=8, Loss=310056.1336465711, Time=86.56402373313904s
Step=9, Loss=310024.3543219692, Time=92.47931170463562s


In [14]:
def NMF_Printout_topics(corpus_file_loc,vocab_file_loc,par_file_loc,results_file_name):
     
    '''
    Visualize Topics
    '''

    #parser = argparse.ArgumentParser()
    #parser.add_argument('--corpus_file', default='data/doc_term_mat.txt', help='term document matrix file')
    #parser.add_argument('--vocab_file', default='data/vocab.txt', help='vocab file')
    #parser.add_argument('--par_file', default='seanmf_results/W.txt', help='model results file')
    #opt = parser.parse_args()

    opt=easydict.EasyDict({'corpus_file':corpus_file_loc,#'data/doc_term_mat_abcnews.txt',
                            'vocab_file':vocab_file_loc,#'data/vocab.txt',
                            'par_file':par_file_loc,#'seanmf_results/W.txt',
                            })

    docs = read_docs(opt.corpus_file)
    vocab = read_vocab(opt.vocab_file)
    n_docs = len(docs)
    n_terms = len(vocab)
    print('n_docs={}, n_terms={}'.format(n_docs, n_terms))

    dt_mat = np.zeros([n_terms, n_terms])
    for itm in docs:
        for kk in itm:
            for jj in itm:
                if kk != jj:
                    dt_mat[int(kk), int(jj)] += 1.0
    print('co-occur done')

    W = np.loadtxt(opt.par_file, dtype=float)
    n_topic = W.shape[1]
    print('n_topic={}'.format(n_topic))

    PMI_arr = []
    n_topKeyword = 10
    for k in range(n_topic):
        topKeywordsIndex = W[:,k].argsort()[::-1][:n_topKeyword]
        PMI_arr.append(calculate_PMI(dt_mat, topKeywordsIndex))
    print('Average PMI={}'.format(np.average(np.array(PMI_arr))))

    index = np.argsort(PMI_arr)

    for k in index:
        print('Topic ' + str(k+1) + ': ', end=' ')
        Topic_temp=k+1
        print(PMI_arr[k], end=' ')
        PMI_temp=PMI_arr[k]

        for w in np.argsort(W[:,k])[::-1][:n_topKeyword]:
            print(vocab[w], end=' ')
        print()
        
    block=pd.DataFrame([]) #initialsing empty 
    val_temp=[] 
    PMI_temp=[]
    Topic_temp=[]
        
    for k in index:

        Topic_temp.append(k+1) #add 1 to value of topic number for acccuracy 

        PMI_temp.append(PMI_arr[k])# append PMI values to list 

        ar_temp=[] # initialise temporary list vocabulary to for each topic  

        for w in np.argsort(W[:,k])[::-1][:n_topKeyword]: #iterating through each topic word 
            ar_temp.append(vocab[w]) #appending each word in topic to list 

        ar_join= ' '.join(ar_temp) #joining all separate words of topic into a single string 

        val_temp.append(ar_join) #appending final completed string into topic list 
    
    block =block.append(pd.DataFrame({'Topic_No':Topic_temp,'PMI_value':PMI_temp,'Topic_string':val_temp}),ignore_index=True) #appending completed topic into a dataframe row 

    block.to_excel(results_file_name)

In [15]:
NMF_Printout_topics('data/doc_term_mat_abcnews.txt','data/vocab.txt','seanmf_results/W.txt','text10.xlsx')

read documents
--------------------------------------------------
read vocabulary
--------------------------------------------------
n_docs=37996, n_terms=5999
co-occur done
n_topic=10
Average PMI=0.9197677994026302
Topic 4:  0.13156471390206848 plan council urge call water job back get report cut 
Topic 10:  0.5572487632731067 new kill crash car die dead swine_flu hospital law road 
Topic 8:  0.5715105838660369 govt urge act qld nt defend vic budget fund nsw 
Topic 9:  0.6111200852497305 man charge face die kill crash miss court murder attack 
Topic 3:  0.8658347545850102 say talk speak nrl_interview tell afl_interview extended_interview minister peter chief 
Topic 1:  0.869598984304244 police probe search miss investigate hunt find suspect death car 
Topic 5:  0.9761847721130219 win aussie lead top take star victory australia award return 
Topic 2:  0.9763791041406219 economic rudd demand economy china mining global trade government bank 
Topic 6:  1.4270489024905233 charge court acc

In [21]:
#Conversion of topics into a dataframe for storage as a CSV file
import pandas as pd

block=pd.DataFrame([]) #initialsing empty 
val_temp=[] 
PMI_temp=[]
Topic_temp=[]


for k in index:

    Topic_temp.append(k+1) #add 1 to value of topic number for acccuracy 
    
    PMI_temp.append(PMI_arr[k])# append PMI values to list 
     
    ar_temp=[] # initialise temporary list vocabulary to for each topic  
    
    for w in np.argsort(W[:,k])[::-1][:n_topKeyword]: #iterating through each topic word 
        ar_temp.append(vocab[w]) #appending each word in topic to list 
    
    ar_join= ' '.join(ar_temp) #joining all separate words of topic into a single string 
    
    val_temp.append(ar_join) #appending final completed string into topic list 
    
block =block.append(pd.DataFrame({'Topic_No':Topic_temp,'PMI_value':PMI_temp,'Topic_string':val_temp}),ignore_index=True) #appending completed topic into a dataframe row 




In [23]:
block.to_excel('2009jan_2_jun_topics_with_text_preprocessing_300_topics.xlsx')

In [22]:
block

,PMI_value,Topic_No,Topic_string
0,0.000000,289,distract panic dampen fielding harm deter mand...
1,0.476488,261,aged_care bega facility karumba desal recyclin...
2,0.486204,101,say time sorry goodbye expert afl_interview ca...
3,0.600738,6,new appoint zealand unveil facility guideline ...
4,0.603865,84,may hurt park underage optus unknown playgroun...
5,0.681749,178,fight vow step save kiwi mundine cambodia way ...
6,0.704586,42,call origin_media spark opposition inquiry act...
7,0.705059,22,take toll control time responsibility stand st...
8,0.738871,286,nt scrymgour welfare croc chamber ant system p...
9,0.755950,24,govt feed ignore pressure must spend trouble e...
